<a href="https://colab.research.google.com/github/vnavya2004/BTP/blob/main/GraphSAGE_SUPERVISED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 32.6 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 31.4 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 36.2 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-9twpt68v
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-9twpt68v
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 241a8c3d018636c116fd1fd7fa2ab9ff3925531e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.6.0-py3-none-any.whl size=11

In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import SAGEConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm
from google.colab import files
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import random

# Load dataset
uploaded = files.upload()
df = pd.read_excel(pd.ExcelFile(list(uploaded.keys())[0]), header=0)
df = df.sample(frac=0.1, random_state=42)

# Graph Preparation
tweets_column = 'tweet'
labels_column = 'label'
NUM_LABELS = len(df[labels_column].unique())
possible_labels = df[labels_column].unique()
label_dict = {possible_label: index for index, possible_label in enumerate(possible_labels)}
df['labels'] = df[labels_column].map(label_dict)

# Split the dataset
df_labeled, df_test = train_test_split(df, stratify=df[labels_column], test_size=0.2)

# Load the XLM-RoBERTa model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModel.from_pretrained('xlm-roberta-base')

def extract_embeddings(texts):
    """Extract embeddings from XLM-RoBERTa model for a list of texts."""
    encoded_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**encoded_inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Take the mean of token embeddings for each input
    return embeddings

def create_graph_data(df, feature_dim=768, top_k=5):
    """
    Creates a graph data object with edges based on similarity of node features.

    Args:
        df (pd.DataFrame): The input data containing tweets and labels.
        feature_dim (int): Dimensionality of node features.
        top_k (int): Number of most similar nodes to connect.

    Returns:
        Data: Graph data object containing node features, edges, and labels.
    """
    # Extract embeddings from XLM-RoBERTa
    texts = df[tweets_column].tolist()
    embeddings = extract_embeddings(texts)

    # Convert embeddings to numpy for similarity computation
    embeddings_np = embeddings.cpu().numpy()

    # Compute cosine similarity between all node embeddings
    similarity_matrix = cosine_similarity(embeddings_np)

    # Create edges based on top_k most similar nodes
    edge_index = []
    for i in range(len(similarity_matrix)):
        # Get indices of the top_k most similar nodes (excluding self)
        similar_nodes = np.argsort(-similarity_matrix[i])[1:top_k+1]
        for j in similar_nodes:
            edge_index.append([i, j])

    # Convert edge index to torch tensor
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # Create graph data
    data = Data(x=embeddings, edge_index=edge_index, y=torch.tensor(df['labels'].values, dtype=torch.long))
    return data

# Create graph data with feature embeddings and similarity-based edges
graph_data_labeled = create_graph_data(df_labeled, feature_dim=768, top_k=5)
graph_data_test = create_graph_data(df_test, feature_dim=768, top_k=5)

def split_data(data, num_subgraphs=10, min_nodes_per_subgraph=10):
    """
    Splits a large graph data object into multiple smaller subgraphs.
    """
    node_indices = list(range(data.num_nodes))
    random.shuffle(node_indices)

    subgraphs = []
    nodes_per_subgraph = max(min_nodes_per_subgraph, len(node_indices) // num_subgraphs)

    for i in range(0, len(node_indices), nodes_per_subgraph):
        subgraph_node_indices = node_indices[i:i + nodes_per_subgraph]
        subgraph_node_indices = torch.tensor(subgraph_node_indices, dtype=torch.long)

        # Extract features and labels for the subgraph nodes
        subgraph_x = data.x[subgraph_node_indices]
        subgraph_y = data.y[subgraph_node_indices]

        # Filter edges that connect the nodes within this subgraph
        mask = torch.isin(data.edge_index[0], subgraph_node_indices) & torch.isin(data.edge_index[1], subgraph_node_indices)
        subgraph_edge_index = data.edge_index[:, mask]

        # Reindex edges for the new subgraph
        node_mapping = {old_idx: new_idx for new_idx, old_idx in enumerate(subgraph_node_indices.tolist())}
        subgraph_edge_index = torch.tensor(
            [[node_mapping[src.item()], node_mapping[dst.item()]] for src, dst in subgraph_edge_index.t()],
            dtype=torch.long
        ).t().contiguous()

        # Create subgraph Data object
        subgraph = Data(x=subgraph_x, edge_index=subgraph_edge_index, y=subgraph_y)
        subgraphs.append(subgraph)

    return subgraphs

# Example usage:
subgraphs_labeled = split_data(graph_data_labeled, num_subgraphs=10, min_nodes_per_subgraph=10)
subgraphs_test = split_data(graph_data_test, num_subgraphs=10, min_nodes_per_subgraph=10)
batch_size = 4

# DataLoaders for batched subgraphs
dataloader_train = DataLoader(subgraphs_labeled, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(subgraphs_test, batch_size=batch_size, shuffle=False)

# Define GNN Model with GraphSAGE
class GNNModel(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GNNModel, self).__init__()
        self.conv1 = SAGEConv(num_node_features, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model
model = GNNModel(num_node_features=768, hidden_channels=64, num_classes=NUM_LABELS)

# Set up the device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set up optimizer and scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
epochs = 10

# Define evaluation metrics
def compute_metrics(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    accuracy = accuracy_score(labels_flat, preds_flat)
    f1 = f1_score(labels_flat, preds_flat, average='weighted')
    precision = precision_score(labels_flat, preds_flat, average='weighted')
    recall = recall_score(labels_flat, preds_flat, average='weighted')
    return accuracy, f1, precision, recall

# Training loop
for epoch in range(1, epochs + 1):
    model.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc=f'Epoch {epoch}', leave=False, disable=False)

    # Train the model on labeled data
    for batch in progress_bar:
        model.zero_grad()
        batch = batch.to(device)

        # Forward pass through the model
        logits = model(batch.x, batch.edge_index)
        loss = F.cross_entropy(logits, batch.y)  # Supervised loss on labeled data

        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item())})

    # Evaluation on the test set
    model.eval()
    predictions, true_labels = [], []
    loss_test_total = 0

    for batch in dataloader_test:
        batch = batch.to(device)
        with torch.no_grad():
            logits = model(batch.x, batch.edge_index)
            loss_test = F.cross_entropy(logits, batch.y)  # Supervised loss on test data
            loss_test_total += loss_test.item()
            logits = logits.detach().cpu().numpy()
            label_ids = batch.y.cpu().numpy()
            predictions.append(logits)
            true_labels.append(label_ids)

    predictions = np.concatenate(predictions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)
    accuracy, f1, precision, recall = compute_metrics(predictions, true_labels)

    # Print evaluation metrics
    print(
        f'\nEpoch {epoch}\n'
        f'Training Loss: {loss_train_total / len(dataloader_train):.3f}\n'
        f'Test Loss: {loss_test_total / len(dataloader_test):.3f}\n'
        f'Accuracy: {accuracy:.3f}\n'
        f'F1 Score: {f1:.3f}\n'
        f'Precision: {precision:.3f}\n'
        f'Recall: {recall:.3f}\n'
    )


Saving Arabic_Depression_10.000_Tweets.xlsx to Arabic_Depression_10.000_Tweets (3).xlsx


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Epoch 1
Training Loss: 0.698
Test Loss: 0.694
Accuracy: 0.510
F1 Score: 0.345
Precision: 0.260
Recall: 0.510




Epoch 2
Training Loss: 0.682
Test Loss: 0.672
Accuracy: 0.645
F1 Score: 0.632
Precision: 0.663
Recall: 0.645




Epoch 3
Training Loss: 0.673
Test Loss: 0.663
Accuracy: 0.710
F1 Score: 0.710
Precision: 0.711
Recall: 0.710



...


Epoch 6
Training Loss: 0.663
Test Loss: 0.651
Accuracy: 0.740
F1 Score: 0.740
Precision: 0.741
Recall: 0.740




Epoch 7
Training Loss: 0.661
Test Loss: 0.650
Accuracy: 0.745
F1 Score: 0.745
Precision: 0.745
Recall: 0.745




Epoch 8
Training Loss: 0.663
Test Loss: 0.649
Accuracy: 0.735
F1 Score: 0.735
Precision: 0.735
Recall: 0.735




Epoch 9
Training Loss: 0.658
Test Loss: 0.649
Accuracy: 0.715
F1 Score: 0.715
Precision: 0.715
Recall: 0.715




Epoch 10
Training Loss: 0.659
Test Loss: 0.649
Accuracy: 0.725
F1 Score: 0.725
Precision: 0.725
Recall: 0.725

